<!-- # 아젠다
# 
# 1.문서의 내용을 읽는다
# 2.문서를 쪼갠다
#     토큰수 초과로 답변을 못할수 있고
#     문서가 길면(인풋이 길면) 답변 생성이 오래걸림
# 3.임베딩 -> 백터 데이터베이스의 저장
# 4.저장한 백터 데이터베이스의 유사도 검색
# 5.검색해서 가져온 문서를 LLM에 질문과 같이 전달 -->

# 1.문서의 내용을 읽는다

In [ ]:
%pip install python-docx

In [ ]:
from docx import Document

document = Document('./tax.docx')

# document

# print(f'document == {dir(document)}')
full_text = ''
for index , paragraph in enumerate(document.paragraphs):
    print(f'document == {paragraph.text}')
    full_text += f'{paragraph.text}\n'

# 2.문서를 쪼갠다

In [ ]:
%pip install tiktoken

In [20]:
import tiktoken


# 프롬프트 문장생성 ctrl + shift + a

def split_text(full_text , chunk_size):
    encoder = tiktoken.encoding_for_model('gpt-3.5-turbo')
    total_encoding = encoder.encode(full_text)
    total_token_count = len(total_encoding)
    
    text_list = []
    
    for i in range(0 , total_token_count , chunk_size):
        chunk = total_encoding[i : i + chunk_size]
        decoded = encoder.decode(chunk)
        text_list.append(decoded)
        
    return text_list
    



In [36]:
chunk_list = split_text(full_text , 1500)

# 3.임베딩

In [25]:

import chromadb

chroma_client = chromadb.Client()


In [27]:
collection_name = 'tax_collection'
taxx_collection = chroma_client.create_collection(collection_name)


In [32]:
from dotenv import load_dotenv
from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction
import os
openai_api_key = os.environ['OPENAI_API_KEY']
openai_embedding = OpenAIEmbeddingFunction(model_name='text-embedding-3-large' , api_key= openai_api_key)

In [44]:
tax_collection = chroma_client.get_or_create_collection(collection_name , embedding_function=openai_embedding)

In [45]:
id_list = []
for index in range(len(chunk_list)):
    id_list.append(f'{index}')

In [40]:
len(id_list)

151

In [42]:
len(chunk_list)





151

In [46]:
tax_collection.add(documents=chunk_list , ids=id_list)